<a href="https://colab.research.google.com/github/Andresf-Asprilla/Modelo-segmentacion-APCIVMAPCAs/blob/main/Utilidades/Escalado_Intensidad_Espaciado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ejecuta el siguiente código una sola vez haciendo clic dentro de la celda y luego presionando Control + Enter.

Antes de ejecutar el siguiente script, debe instalar las siguientes dependencias:
```
!pip install -q "monai-weekly[gdown, nibabel, tqdm, itk]"
!pip install SimpleITK
```



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import SimpleITK as sitk
import nibabel as nib
import monai.transforms as tf
import os

def NormalizacionyEspaciado(ruta_directorio, ruta_salida, nuevo_espaciado):
    """
     Cambia el espaciado de todas las imágenes en un directorio sin modificar sus dimensiones ni el tamaño de las estructuras.
    :param ruta_directorio: Ruta al directorio que contiene los archivos NIfTI de entrada.
    :param ruta_salida: Ruta al directorio donde se guardarán los archivos procesados.
    :param nuevo_espaciado: Nuevo espaciado como lista [x, y, z].
    """
    os.makedirs(ruta_salida, exist_ok=True)

    for archivo in os.listdir(ruta_directorio):
        if archivo.endswith('.nii') or archivo.endswith('.nii.gz'):
            ruta_entrada = os.path.join(ruta_directorio, archivo)
            ruta_salida_archivo = os.path.join(ruta_salida, archivo)

            # Leer imagen con SimpleITK
            imagen = sitk.ReadImage(ruta_entrada)

            # Mostrar info original
            print(f"\nProcesando: {archivo}")
            print(f"Espaciado original: {imagen.GetSpacing()}")
            print(f"Tamaño: {imagen.GetSize()}")
            print(f"Origen: {imagen.GetOrigin()}")

            # Cambiar solo el espaciado
            imagen.SetSpacing(nuevo_espaciado)

            # Guardar temporalmente
            sitk.WriteImage(imagen, ruta_salida_archivo)
            print(f"Espaciado actualizado a: {nuevo_espaciado}")

            # Normalización de intensidad
            nifti_imagen = nib.load(ruta_salida_archivo)
            datos_imagen = nifti_imagen.get_fdata()
            affine = nifti_imagen.affine

            escalar_intensidad = tf.ScaleIntensity()
            imagen_normalizada = escalar_intensidad(datos_imagen).numpy()

            nib.save(nib.Nifti1Image(imagen_normalizada, affine), ruta_salida_archivo)
            print("Normalización de intensidad aplicada y guardada.")


Mounted at /content/drive


In [ ]:
# Uso para un directorio
ruta_directorio = "/content/drive/MyDrive/Colab Notebooks/Imagenes_redimencionadas_Respaciadas/ImagesEscalarRedimecionar"# cambia esta direccion por la carpeta donde se encuentra ubicadas las imagenes dicom en tu drive.
ruta_salida = "/content/drive/MyDrive/Colab Notebooks/Imagenes_redimencionadas_Respaciadas/EscaladasRedimecionadas" # Cambia esta dirección por la carpeta  donde deseas que las imágenes reescaladas y espacadas se almacenen.
nuevo_espaciado = [1.0, 1.0, 1.0]  # Espaciado deseado en mm

NormalizacionyEspaciado(ruta_directorio, ruta_salida, nuevo_espaciado)
